In [5]:
import numpy as np
# def linespace (delta):#
#     # 生成气象版本的linspace,生成的是经纬度格网的顶点，经度从-180到180-deltaLon 维度从-90到90 所以对于0.25°的格网来说，经度是从-180到179.75 维度是从-90到89.75
#     # 纬度格网数为721 经度格网数为1440
#     lon = np.arange(-180+delta,180,delta)
#     # 按照地理坐标系的规则 维度从下到上是增大
#     lat = np.arange(90-delta,-90,-delta)
#     print('lon',lon.shape )
#     # print('lat',lat )
#     # print('lat',lat.shape )
    

#     return lon,lat

def linespace (delta):#
    # 生成气象版本的linspace,生成的是经纬度格网的顶点，经度从-180到180-deltaLon 维度从-90到90 所以对于0.25°的格网来说，经度是从-180到179.75 维度是从-90到89.75
    # 纬度格网数为721 经度格网数为1440
    lon = np.arange(0,360,delta)
    # 按照地理坐标系的规则 维度从下到上是增大
    lat = np.arange(90,-90-delta,-delta)
    print('lon',lon.shape)
    print('lat',lat.shape)
    return lon,lat
a= linespace(0.5)

lon (720,)
lat (361,)


In [3]:
import xarray as xr
data_path = '/media/dls/WeatherData/ENS10/meanstd_pl'
ds_mean = xr.open_dataset(f"{data_path}/ENS10_pl_mean.nc", chunks={"time": 10})


In [6]:
ds_mean_array=ds_mean.drop_vars('time_bnds') 
ds_mean_array.to_array()[0,0,...].values.shape

(2, 361, 720)

In [6]:
ds_mean

<xarray.Dataset>
Dimensions:    (time: 2099, bnds: 2, lon: 720, lat: 361, plev: 2)
Coordinates:
  * time       (time) datetime64[ns] 1998-01-03 1998-01-06 ... 2017-12-29
  * lon        (lon) float64 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * lat        (lat) float64 90.0 89.5 89.0 88.5 ... -88.5 -89.0 -89.5 -90.0
  * plev       (plev) float64 5e+04 8.5e+04
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(10, 2), meta=np.ndarray>
    Z          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    T          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    Q          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    W          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    D          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    U          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    V          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
Attributes:
    CDI:                       Climate Data Interface version 2.0.4 (https://...
    Conventions:               CF-1.6
    institution:               European Centre for Medium-Range Weather Forec...
    frequency:                 day
    history:                   Sat Sep 16 18:25:23 2023: cdo -P 16 -f nc4 -z ...
    CDO:                       Climate Data Operators version 2.0.4 (https://...
    cdo_openmp_thread_number:  16

In [9]:
ds_mean.sel(plev =80000)

KeyError: "not all values found in index 'plev'. Try setting the `method` keyword argument (example: method='nearest')."

In [2]:
ds_mean_avg = ds_mean.mean(dim='time')
ds_mean_std = ds_mean.std(dim='time')



In [ ]:
ds_mean_std.to_netcdf(f"{data_path}/ENS10_pl_mean_std.nc")
ds_mean_avg.to_netcdf(f"{data_path}/ENS10_pl_mean_avg.nc")


In [3]:
import torch 
time= 2992
lon=720
lat=361
plev=2
lon*lat
# x =torch.randn(time,plev,lon,lat)

259920

In [4]:
import torch.nn as nn
# unfold = nn.Unfold(kernel_size=(3, 3),padding=1)
input = torch.randn(plev,time, lat, lon)#time纬度滑块展开
# output = unfold(input)
output = torch.var(input, dim=1)
# each patch contains 30 values (2x3=6 vectors, each of 5 channels)
# 4 blocks (2x3 kernels) in total in the 3x4 input
output.size()

# # Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)
# inp = torch.randn(1, 3, 10, 12)
# w = torch.randn(2, 3, 4, 5)
# inp_unf = torch.nn.functional.unfold(inp, (4, 5))
# out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
# out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))
# # or equivalently (and avoiding a copy),
# # out = out_unf.view(1, 2, 7, 8)
# (torch.nn.functional.conv2d(inp, w) - out).abs().max()

torch.Size([2, 361, 720])

In [6]:
import xarray as xr
data_path = '/media/dls/WeatherData/ENS10/meanstd'
ds_sfc = xr.open_dataset(f"/media/dls/WeatherData/ENS10/meanstd/ENS10_sfc_mean.nc", chunks={"time": 10})



In [7]:
ds_sfc

<xarray.Dataset>
Dimensions:    (time: 2099, bnds: 2, lon: 720, lat: 361)
Coordinates:
  * time       (time) datetime64[ns] 1998-01-03 1998-01-06 ... 2017-12-29
  * lon        (lon) float64 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * lat        (lat) float64 90.0 89.5 89.0 88.5 ... -88.5 -89.0 -89.5 -90.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(10, 2), meta=np.ndarray>
    SSTK       (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    TCW        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    TCWV       (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    CP         (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    MSL        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    TCC        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    U10M       (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    V10M       (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    T2M        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    TP         (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    SKT        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
Attributes:
    CDI:                       Climate Data Interface version 2.0.4 (https://...
    Conventions:               CF-1.6
    institution:               European Centre for Medium-Range Weather Forec...
    frequency:                 day
    history:                   Tue Aug 15 19:49:08 2023: cdo -P 16 -f nc4 -z ...
    CDO:                       Climate Data Operators version 2.0.4 (https://...
    cdo_openmp_thread_number:  16

In [4]:
import numpy as np  
path = '/media/dls/WeatherData/ENS10/data/t2m/img/t2m_19980103.npy'
labelpath = '/media/dls/WeatherData/ENS10/data/t2m/label/t2mlabel_19980103.npy'

data =np.load(labelpath,allow_pickle=True)


In [12]:
data

array(<PixelData(

    META INFORMATION

    DATA FIELDS
    data: tensor([[[251.8372, 251.8372, 251.8372,  ..., 251.8372, 251.8372, 251.8372],
                 [250.6165, 250.6223, 250.6321,  ..., 250.5989, 250.6067, 250.6145],
                 [250.4153, 250.4095, 250.4036,  ..., 250.4544, 250.4407, 250.4270],
                 ...,
                 [249.2278, 249.2044, 249.1790,  ..., 249.2903, 249.2688, 249.2493],
                 [249.0559, 249.0442, 249.0305,  ..., 249.0930, 249.0794, 249.0657],
                 [248.6223, 248.6223, 248.6223,  ..., 248.6223, 248.6223, 248.6223]]])
) at 0x7f4cec82fa30>, dtype=object)

In [4]:
np.load(path,allow_pickle=False)


array([[[2.7146085e+02, 2.7146085e+02, 2.7146085e+02, ...,
         2.7146085e+02, 2.7146085e+02, 2.7146085e+02],
        [2.7146045e+02, 2.7146045e+02, 2.7146045e+02, ...,
         2.7146045e+02, 2.7146045e+02, 2.7146045e+02],
        [2.7146045e+02, 2.7146045e+02, 2.7146045e+02, ...,
         2.7146045e+02, 2.7146045e+02, 2.7146045e+02],
        ...,
        [2.7316260e+02, 2.7316357e+02, 2.7316455e+02, ...,
         2.7316064e+02, 2.7316162e+02, 2.7316162e+02],
        [2.7321436e+02, 2.7321533e+02, 2.7321631e+02, ...,
         2.7321045e+02, 2.7321240e+02, 2.7321338e+02],
        [2.7360693e+02, 2.7360693e+02, 2.7360693e+02, ...,
         2.7360693e+02, 2.7360693e+02, 2.7360693e+02]],

       [[4.7841840e-04, 4.7841840e-04, 4.7841840e-04, ...,
         4.7841840e-04, 4.7841840e-04, 4.7841840e-04],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.000